<a href="https://colab.research.google.com/github/marcosfs2006/rede-rpps/blob/master/Analise_Fundos_Investimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----

# **FUNDOS DE INVESTIMENTOS**

----

O dados do DAIR utilizados neste notebook são oriundos dos aquivos disponibilizados pela SPrev em seu [site de dados abertos](http://www.previdencia.gov.br/dados-abertos/estatisticas-e-informacoes-dos-rpps/).

 Os arquivos importados são os seguintes:

* `carteira2018_atualizacao_ de_ fev_2020.xlsx`   
* `carteira2019_atualizacao_ de_ fev_2020_B.xlsx`   

A importação desses dados e geração do conjunto de dados `dair_DtRef_FEV2020.Rds` disponibilizado neste repositório é feita utilizando-se o código contido no _script_ `importacao_dados_dair.R` também disponíbilizado neste repositório.

Para conhecer um pouco mais sobre fundos de investimentos, consulte a página site http://www.cvm.gov.br/menu/regulados/fundos/sobre.html

<br>
<br>

---

# **PARTE I - CONSTRUÇÃO DE UMA BASE DADOS DE FUNDOS DE INVESTIMENTOS**

---

<br>

In [0]:
# Anexar o repositório do GitHub contendo os dados...
system('git clone https://github.com/marcosfs2006/rede-rpps.git')
setwd("/content/rede-rpps")

In [0]:
# Carregar os pacotes necessários...
options(scipen = 999, message = FALSE)
library(readr)
library(dplyr)
library(ggplot2)
library(stringr)
library(readxl)
library(lubridate)
library(purrr)

In [83]:
# Importar os dados do DAIR...

dair  <- readRDS("dair_DtRef_FEV2020.Rds") %>% 
              mutate(enquad_rpps_pad = str_extract(tipo_ativo, "Art.*$"),
                     enquad_rpps_pad = toupper(gsub("[[:punct:]]| |º", "", enquad_rpps_pad))) 

glimpse(dair)

Rows: 827,213
Columns: 16
$ cnpj               <dbl> 63606479000124, 63606479000124, 63606479000124, 63…
$ uf                 <chr> "AC", "AC", "AC", "AC", "AC", "AC", "AC", "AC", "A…
$ ente               <chr> "Governo do Estado do Acre", "Governo do Estado do…
$ competencia        <date> 2018-01-01, 2018-01-01, 2018-01-01, 2018-01-01, 2…
$ segmento           <chr> "Disponibilidades Financeiras", "Disponibilidades …
$ tipo_ativo         <chr> "Disp", "Disp", "Disp", "FI 100% títulos TN", "Dis…
$ limite_resol_cmn   <dbl> NA, NA, NA, 99.99, NA, NA, NA, NA, 99.99, NA, NA, …
$ ident_ativo        <chr> "3550  60801", "3550  5738X", "3550  83070", "1132…
$ nm_ativo           <chr> "1 Banco do Brasil S.A.", "1 Banco do Brasil S.A."…
$ qtd_quotas         <dbl> 1.0, 1.0, 1.0, 252362.9, 1.0, 1.0, 1.0, 1.0, 14443…
$ vlr_atual_ativo    <dbl> 0.000000, 14391809.910000, 0.000000, 2.330250, 0.0…
$ vlr_total_atual    <dbl> 0.0, 14391809.9, 0.0, 588068.6, 0.0, 0.0, 0.0, 0.0…
$ perc_recursos_rpps <dbl>

A base do DAIR contem todos os ativos nos quais o cada RPPS aplica seus recursos.

### **Base de Fundos de Investimentos**

Para o nosso estudo, apenas os fundos de investimento são de interesse.

 No código a seguir vamos aplicar um filtro a essa base de dados de forma a só ficarmos com os fundos de investimentos.

In [84]:
# filtrar a base para pegar apenas os registros cujos ativos sejam fundos de investimentos
dair_fundos <- dair %>% 
              filter(competencia >= as.Date("2018-04-01"),  # Pega dados posteriores a Abr/2018
                     grepl("\\d{14}", ident_ativo))         # Pega só fundos

glimpse(dair_fundos)

Rows: 491,572
Columns: 16
$ cnpj               <dbl> 63606479000124, 63606479000124, 63606479000124, 63…
$ uf                 <chr> "AC", "AC", "AC", "AC", "AC", "AC", "AC", "AC", "A…
$ ente               <chr> "Governo do Estado do Acre", "Governo do Estado do…
$ competencia        <date> 2018-04-01, 2018-05-01, 2018-06-01, 2018-07-01, 2…
$ segmento           <chr> "Renda Fixa", "Renda Fixa", "Renda Fixa", "Renda F…
$ tipo_ativo         <chr> "FI 100% títulos TN - Art. 7º, I, b", "FI 100% tít…
$ limite_resol_cmn   <dbl> 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, …
$ ident_ativo        <chr> "11328882000135", "11328882000135", "1132888200013…
$ nm_ativo           <chr> "BB PREVIDENCIÁRIO RENDA FIXA IRF M1", "BB PREVIDE…
$ qtd_quotas         <dbl> 321991.422378, 988787.658155, 5770005.412829, 7141…
$ vlr_atual_ativo    <dbl> 2.368637, 2.372936, 2.384796, 2.400908, 2.410399, …
$ vlr_total_atual    <dbl> 762680.84, 2346329.90, 13760284.67, 17146336.11, 3…
$ perc_recursos_rpps <dbl>

### **Quantidade de RPPS investindo em cada FI**

Vamos calcular quantos RPPS possuem investimentos em cada FI.

In [85]:
# Quantos RPPS investem em cada um dos fundos de investimentos existentes na base de dados
rpps_invest <- dair_fundos %>%
                group_by(ident_ativo) %>%
                distinct(cnpj) %>%
                count() %>%
                arrange(desc(n)) %>%
                rename(qtd_rpps_investindo = n)

head(rpps_invest, 10)

ident_ativo,qtd_rpps_investindo
<chr>,<int>
10740670000106,1399
11328882000135,1258
10740658000193,1006
11060913000110,978
14386926000171,977
13322205000135,927
13077418000149,845
07111384000169,788
07442078000105,768


Essa variável será inserida posteriormente na base que estamos montando.

### **Fundos Vedados**

Agora vamos importar a base de fundos vedados.

In [86]:
# Importar base de fundos vedados
fundos_vedados      <- readRDS("fundos_vedados.Rds") 

names(fundos_vedados) <- c("cnpj_fundo", "nm_fundo", "classe1", "classe2", "tx_adm",
                           "administrador", "gestor", "carencia", "conversao_cotas",
                           "disp_rec_resgatado", "tx_saida", "motivo")

fundos_vedados <- fundos_vedados %>% 
                          mutate(cnpj_fundo = gsub("[[:punct:]]", "", cnpj_fundo)) %>% 
                          map_df(as.character)

glimpse(fundos_vedados)

Rows: 138
Columns: 12
$ cnpj_fundo         <chr> "00828035000113", "01107772000190", "0137595400014…
$ nm_fundo           <chr> "FATOR MAX CORPORATIVO FUNDO DE INVESTIMENTO DE RE…
$ classe1            <chr> "FI", "FI", "FI", "FI", "FI", "FI", "FI", "FI", "F…
$ classe2            <chr> "Fundo de Renda Fixa", "Fundo de Renda Fixa", "Fun…
$ tx_adm             <chr> "0", "1", "2", "1", "2", "2", "2", "0", "0", "2", …
$ administrador      <chr> "BANCO FATOR S/A", "CONCÓRDIA S.A. CVMCC", "PLANNE…
$ gestor             <chr> "FAR FATOR ADMINISTRAÇÃO DE RECURSOS LTDA", "CONCÓ…
$ carencia           <chr> "Não há.", "Não há.", "Não há.", "Não há.", "2 ano…
$ conversao_cotas    <chr> "D+0", "D+0", "D+0", "D+0", "D+0", "(vide regulame…
$ disp_rec_resgatado <chr> "D+0", "D+0", "D+0", "D+1", "D+10", "(vide regulam…
$ tx_saida           <chr> "0", "0", "0", "0", "0", "2", "0", "0", "0", "2", …
$ motivo             <chr> "Ativo Vedado / CRI/CRA /", "Ativo Vedado / FIDC V…


In [87]:
# Motivos polos quais os fundos são considerados vedados...
unique(trimws(unlist(str_split(fundos_vedados$motivo, "/"))))


[1] "Ativo Vedado"                                 
 [2] "CRI"                                          
 [3] "CRA"                                          
 [4] ""                                             
 [5] "FIDC Vedado"                                  
 [6] "FIP Vedado"                                   
 [7] "FII Vedado"                                   
 [8] "FII - Negociação 60%"                         
 [9] "Investidor Profissional"                      
[10] "FIDC Vedado - Classe Única"                   
[11] "FIP Vedado, Investidor Profissional"          
[12] "FII - Negociação 60%, Investidor Profissional"

### **Enquadramento dos Fundos de Investimentos**

Agora vamos importar a base de dados de enquadramento dos fundos de investimento. 

Essa base de dados informa qual a classificação do fundo de investimento com base na Resolução CMN 3922/10, segundo a SPrev.

Muitas vezes essa classificação (feita pela SPrev) diverge da classificação realizada pelo RPPS e presente na base de dados do DAIR.

In [88]:
# Importa a base de enquadramento
enquadramento_sprev <- readRDS("enquadramento_sprev_12-03-2020.Rds") %>% 
                          mutate(cnpj = str_remove_all(cnpj, "[[:punct:]]"),
                                 enquad_sprev_pad = str_replace_all(enquad_sprev, c("Artigo" = "Art", "Inciso" = "")),
                                 enquad_sprev_pad = toupper(gsub("[[:punct:]]| |º", "", enquad_sprev_pad)),
                                 cnpj_admin = str_extract(gsub("[[:punct:]]", "", cnpj_admin),  "^\\d{8}"),
                                 cnpj_gestor= str_extract(gsub("[[:punct:]]", "", cnpj_gestor), "^\\d{8}"))

glimpse(enquadramento_sprev)

Rows: 651
Columns: 6
$ nm_fundo         <chr> "BB RENDA FIXA REFERENCIADO DI ÁGIL FUNDO DE INVESTI…
$ cnpj             <chr> "00756851000169", "00812433000141", "00822059000165"…
$ cnpj_admin       <chr> "30822936", "62318407", "30822936", "60701190", "003…
$ cnpj_gestor      <chr> "30822936", "07237373", "30822936", "60701190", "003…
$ enquad_sprev     <chr> "Artigo 7º, Inciso IV, 'a'", "Artigo 7º, Inciso IV, …
$ enquad_sprev_pad <chr> "ART7IVA", "ART7IVA", "ART8IA", "ART7IVA", "ART7IVA"…


### **Artigo 15 da Resolução CMN 3922/10**

Agora vamos importar os dados das empresas autorizadas a atuar como administradoras ou gestoras de fundos de investimentos. 

Se um fundo de investimento tiver por administrador e gestor uma empresa fora dessa base de dados (disponibilizada pela SPrev) esse fundo de investimento não pode receber recursos dos RPPS.

In [89]:
# Importar dados dos Administradore e gestores autorizados a distribuir fundos de investimentos

art15 <- read_delim("Art15_ResolucaoCMN.txt", delim="|", trim_ws = TRUE) %>%
              mutate(cnpj = gsub("[[:punct:]]", "", cnpj))

glimpse(art15)

Parsed with column specification:
cols(
  cnpj = col_character(),
  instituicao_financeira = col_character(),
  conglomerado = col_character(),
  pagina = col_double()
)



Rows: 43
Columns: 4
$ cnpj                   <chr> "01023570", "01181521", "01522368", "01638542"…
$ instituicao_financeira <chr> "BANCO RABOBANK INTERNATIONAL BRASIL S.A.", "B…
$ conglomerado           <chr> "BCO RABOBANK INTL BRASIL S.A.", "BCO COOPERAT…
$ pagina                 <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…


### **Relação dos fundos sob investigação**

Vamos agora importar dos dados relativos a fundos de investimento que, de alguma forma, foram objeto de investigação pela Polícia Federal, SPrev ou penalizadas pela Previc ou CVM. 

Esta é uma base de dados que estará sendo constantemente ampliada. 

In [90]:
fundos_investigados <- read_excel("/content/rede-rpps/the-black-list.xlsx")

head(fundos_investigados)

cnpj_fundo,nome_fundo,obs
<chr>,<chr>,<chr>
12.845.801/0001-37,TOWER BRIDGE RF FI IMA-B,Proc. 225.366-4/17
15.153.656/0001-11,FI RF MONTE CARLO INSTITUCIONAL IMA-B,Proc. 225.366-4/17
13.651.947/0001-04,TOTEM FI RENDA FIXA,Proc. 225.366-4/17
09.315.625/0001-17,FI SECURITY REFERENCIADO DI LP CP,Proc. 225.366-4/17
11.989.256/0001-90,FIDC MULTISETORIAL MASTER II,Proc. 225.366-4/17
11.351.413/0001-37,CORAL FIDC MULTISETORIAL,Proc. 225.366-4/17


Agora vamos limpar um pouco a base e obter a relação dos fundos investigados.



In [0]:
fundos_investigados <- fundos_investigados %>%
                          filter(obs != "FUNDOS VEDADOS") %>%
                          mutate(cnpj_fundo = str_remove_all(cnpj_fundo, "[[:punct:]]| ")) %>%
                          pull(cnpj_fundo) %>%
                          unique()

### **Base de Dados - Início da Construção**

Importados os dados, vamos agora à relação dos fundos de investimentos nos quais os RPPS mantém recursos aplicados. 

A relação será criada considerando os fundos de investimentos existentes na carteira dos RPPS no mês mais recente disponível na base de dados do DAIR.

 É importante frisar que nem todos os RPPS encaminham o DAIR à SPrev e, portanto, não estão na base de dados. 

In [0]:
# Obter a relação de Fundos de Investimentos nos quais os RPPS investem seus recursos
relacao_fundos <- dair_fundos %>%
                      group_by(cnpj) %>%
                      filter(competencia == max(competencia)) %>%
                      ungroup() %>% 
                      distinct(ident_ativo, .keep_all = TRUE) %>%
                      arrange(ident_ativo) %>%
                      select(ident_ativo, nm_ativo)

In [93]:
glimpse(relacao_fundos)

Rows: 644
Columns: 2
$ ident_ativo <chr> "00071477000168", "00360305000104", "00756851000169", "00…
$ nm_ativo    <chr> "BB RF CP AUTOMATICO", "CAIXA FI BRASIL IRF-M 1 TP RF", "…


Obtida a relação de fundos nos quais os RPPS investem seus recursos, vamos começar a adicioar a esta listagem algumas variáveis contidas nas bases de dados anteriormente importadas.

A primeira, será uma indicação se o fundo está na listagem de fundos vedados ou não.

In [0]:
relacao_fundos <- relacao_fundos %>%
                    mutate(eh_fundo_vedado = ifelse(ident_ativo %in% fundos_vedados$cnpj_fundo, 1, 0))

In [95]:
glimpse(relacao_fundos)

Rows: 644
Columns: 3
$ ident_ativo     <chr> "00071477000168", "00360305000104", "00756851000169",…
$ nm_ativo        <chr> "BB RF CP AUTOMATICO", "CAIXA FI BRASIL IRF-M 1 TP RF…
$ eh_fundo_vedado <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…


Qual a proporção de fundos vedados existente na carteira dos RPPS?

In [96]:
relacao_fundos %>% count(eh_fundo_vedado)

prop.table(table(relacao_fundos$eh_fundo_vedado))

eh_fundo_vedado,n
<dbl>,<int>
0,546
1,98



        0         1 
0.8478261 0.1521739 

Quase 16% dos fundos são fundos vedados.

Agora vamos juntar à relação de fundos a sua classificação (variável `enquad_sprev`) contida na base de dados `enquadramento_sprev`.

In [0]:
relacao_fundos <- relacao_fundos %>%
                      left_join(enquadramento_sprev, by = c("ident_ativo" = "cnpj"))

In [98]:
glimpse(relacao_fundos)

Rows: 644
Columns: 8
$ ident_ativo      <chr> "00071477000168", "00360305000104", "00756851000169"…
$ nm_ativo         <chr> "BB RF CP AUTOMATICO", "CAIXA FI BRASIL IRF-M 1 TP R…
$ eh_fundo_vedado  <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ nm_fundo         <chr> NA, NA, "BB RENDA FIXA REFERENCIADO DI ÁGIL FUNDO DE…
$ cnpj_admin       <chr> NA, NA, "30822936", "62318407", "30822936", "6070119…
$ cnpj_gestor      <chr> NA, NA, "30822936", "07237373", "30822936", "6070119…
$ enquad_sprev     <chr> NA, NA, "Artigo 7º, Inciso IV, 'a'", "Artigo 7º, Inc…
$ enquad_sprev_pad <chr> NA, NA, "ART7IVA", "ART7IVA", "ART8IA", "ART7IVA", "…


Agora vamos criar uma coluna/variável na base para indicar se o fundo de investimento atende ao art. 15 da Resolução CMN 3922/10 ou não.

In [99]:
relacao_fundos <- relacao_fundos %>%
    mutate(atende_art15 = ifelse(cnpj_admin %in% art15$cnpj | cnpj_gestor %in% art15$cnpj, 1, 0))
                          
head(relacao_fundos)

ident_ativo,nm_ativo,eh_fundo_vedado,nm_fundo,cnpj_admin,cnpj_gestor,enquad_sprev,enquad_sprev_pad,atende_art15
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
00071477000168,BB RF CP AUTOMATICO,0,NA,NA,NA,NA,NA,0
00360305000104,CAIXA FI BRASIL IRF-M 1 TP RF,0,NA,NA,NA,NA,NA,0
00756851000169,BB ÁGIL FIC RENDA FIXA REFERENCIADO DI,0,BB RENDA FIXA REFERENCIADO DI ÁGIL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1
00812433000141,BNB AUTOMATICO FI RF CURTO PRAZO,0,BNB AUTOMÁTICO FUNDO DE INVESTIMENTO RENDA FIXA CURTO PRAZO,62318407,07237373,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1
00822059000165,BB AÇÕES IBOVESPA ATIVO FIC,0,BB AÇÕES IBOVESPA ATIVO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 8º, Inciso I, 'a'",ART8IA,1
00832435000100,ITAÚ INSTITUCIONAL RENDA FIXA REFERENCIADO DI - FI,0,ITAÚ INSTITUCIONAL RENDA FIXA REFERENCIADO DI - FUNDO DE INVESTIMENTO,60701190,60701190,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1


Na tabela acima é preciso fazer uma correção. Se o valor do CNPJ do Administrador ou Gestor for `NA` então a variável `atende_art15` deve ser `NA` e não zero, como está. Vamos fazer a correção.

In [100]:
relacao_fundos <- relacao_fundos %>%
                    mutate(atende_art15 = ifelse(is.na(cnpj_admin) | is.na(cnpj_gestor), NA, atende_art15))

head(relacao_fundos)

ident_ativo,nm_ativo,eh_fundo_vedado,nm_fundo,cnpj_admin,cnpj_gestor,enquad_sprev,enquad_sprev_pad,atende_art15
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
00071477000168,BB RF CP AUTOMATICO,0,NA,NA,NA,NA,NA,NA
00360305000104,CAIXA FI BRASIL IRF-M 1 TP RF,0,NA,NA,NA,NA,NA,NA
00756851000169,BB ÁGIL FIC RENDA FIXA REFERENCIADO DI,0,BB RENDA FIXA REFERENCIADO DI ÁGIL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1
00812433000141,BNB AUTOMATICO FI RF CURTO PRAZO,0,BNB AUTOMÁTICO FUNDO DE INVESTIMENTO RENDA FIXA CURTO PRAZO,62318407,07237373,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1
00822059000165,BB AÇÕES IBOVESPA ATIVO FIC,0,BB AÇÕES IBOVESPA ATIVO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 8º, Inciso I, 'a'",ART8IA,1
00832435000100,ITAÚ INSTITUCIONAL RENDA FIXA REFERENCIADO DI - FI,0,ITAÚ INSTITUCIONAL RENDA FIXA REFERENCIADO DI - FUNDO DE INVESTIMENTO,60701190,60701190,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1


Agora vamos importar dados cadastrais dos fundos de investimentos. Os dados estão disponíveis no site de dados abertos da CVM: http://dados.cvm.gov.br/dataset/fie-cad



In [101]:
cadastro <- read_csv2("http://dados.cvm.gov.br/dados/FIE/CAD/DADOS/inf_cadastral_fie.csv",
                       locale = locale(encoding = "latin1"))

Using ',' as decimal and '.' as grouping mark. Use read_delim() for more control.

Parsed with column specification:
cols(
  TP_FUNDO = col_character(),
  CNPJ_FUNDO = col_character(),
  DENOM_SOCIAL = col_character(),
  DT_REG = col_date(format = ""),
  DT_CONST = col_date(format = ""),
  DT_CANCEL = col_date(format = ""),
  SIT = col_character(),
  DT_INI_SIT = col_date(format = ""),
  DT_INI_EXERC = col_date(format = ""),
  DT_FIM_EXERC = col_date(format = ""),
  CNPJ_ADMIN = col_character(),
  ADMIN = col_character(),
  PF_PJ_GESTOR = col_character(),
  CPF_CNPJ_GESTOR = col_character(),
  GESTOR = col_character(),
  CNPJ_AUDITOR = col_character(),
  AUDITOR = col_character()
)



In [102]:
glimpse(cadastro)

Rows: 16,142
Columns: 17
$ TP_FUNDO        <chr> "F.I.I.", "F.I.I.", "F.I.I.", "F.I.I.", "F.I.I.", "F.…
$ CNPJ_FUNDO      <chr> "19.878.616/0001-98", "19.419.994/0001-03", "19.213.2…
$ DENOM_SOCIAL    <chr> "17007 NAÇÕES FUNDO DE INVESTIMENTO IMOBILIÁRIO", "25…
$ DT_REG          <date> 2019-12-16, 2013-12-27, 2017-12-29, 2017-12-29, 2019…
$ DT_CONST        <date> 2019-12-13, 2013-12-10, 2013-10-11, 2014-10-10, 2018…
$ DT_CANCEL       <date> NA, NA, NA, NA, NA, 2014-06-27, NA, NA, 2018-03-22, …
$ SIT             <chr> "EM FUNCIONAMENTO NORMAL", "EM FUNCIONAMENTO NORMAL",…
$ DT_INI_SIT      <date> 2019-12-27, 2013-12-27, 2017-12-29, 2017-12-29, 2019…
$ DT_INI_EXERC    <date> 2020-01-01, 2020-01-01, 2020-01-01, 2020-01-01, 2020…
$ DT_FIM_EXERC    <date> 2020-12-31, 2020-12-31, 2020-12-31, 2020-12-31, 2020…
$ CNPJ_ADMIN      <chr> "45.246.410/0001-55", "36.113.876/0001-91", "03.317.6…
$ ADMIN           <chr> "PLURAL S.A. BANCO MULTIPLO", "OLIVEIRA TRUST DTVM S.…
$ PF_PJ_GESTOR    <chr> "PJ

In [103]:
head(cadastro, 3)

TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_EXERC,DT_FIM_EXERC,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR
<chr>,<chr>,<chr>,<date>,<date>,<date>,<chr>,<date>,<date>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
F.I.I.,19.878.616/0001-98,17007 NAÇÕES FUNDO DE INVESTIMENTO IMOBILIÁRIO,2019-12-16,2019-12-13,NA,EM FUNCIONAMENTO NORMAL,2019-12-27,2020-01-01,2020-12-31,45.246.410/0001-55,PLURAL S.A. BANCO MULTIPLO,PJ,22.119.959/0001-83,BRPP GESTÃO DE PRODUTOS ESTRUTURADOS LTDA.,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES
F.I.I.,19.419.994/0001-03,2509 FUNDO DE INVESTIMENTO IMOBILIÁRIO - FII,2013-12-27,2013-12-10,NA,EM FUNCIONAMENTO NORMAL,2013-12-27,2020-01-01,2020-12-31,36.113.876/0001-91,OLIVEIRA TRUST DTVM S.A.,PJ,36.113.876/0001-91,OLIVEIRA TRUST DTVM S.A.,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES
F.I.I.,19.213.231/0001-01,AB CAPITAL FUNDO DE INVESTIMENTO IMOBILIÁRIO,2017-12-29,2013-10-11,NA,EM FUNCIONAMENTO NORMAL,2017-12-29,2020-01-01,2020-12-31,03.317.692/0001-94,FINAXIS CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.,PJ,09.146.756/0001-18,JERA CAPITAL GESTAO DE RECURSOS LTDA,10.830.108/0001-65,GRANT THORNTON AUDITORES INDEPENDENTES


Agora podemos juntar à relação de fundos os dados cadastrais de cada fundo. Mas antes é necessário relizar uma pequena limpeza na variável `CNPJ_FUNDO`.

In [0]:
cadastro <- cadastro %>%
                mutate(CNPJ_FUNDO = str_remove_all(CNPJ_FUNDO, "[[:punct:]]"))

In [0]:
relacao_fundos <- relacao_fundos %>%
                      left_join(cadastro, by=c("ident_ativo" = "CNPJ_FUNDO"))

In [106]:
glimpse(relacao_fundos)

Rows: 656
Columns: 25
$ ident_ativo      <chr> "00071477000168", "00360305000104", "00756851000169"…
$ nm_ativo         <chr> "BB RF CP AUTOMATICO", "CAIXA FI BRASIL IRF-M 1 TP R…
$ eh_fundo_vedado  <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ nm_fundo         <chr> NA, NA, "BB RENDA FIXA REFERENCIADO DI ÁGIL FUNDO DE…
$ cnpj_admin       <chr> NA, NA, "30822936", "62318407", "30822936", "6070119…
$ cnpj_gestor      <chr> NA, NA, "30822936", "07237373", "30822936", "6070119…
$ enquad_sprev     <chr> NA, NA, "Artigo 7º, Inciso IV, 'a'", "Artigo 7º, Inc…
$ enquad_sprev_pad <chr> NA, NA, "ART7IVA", "ART7IVA", "ART8IA", "ART7IVA", "…
$ atende_art15     <dbl> NA, NA, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, NA, 1, 1, 1…
$ TP_FUNDO         <chr> "FACFIF", NA, "FACFIF", "FIF", NA, "FIF", "FIF", "FI…
$ DENOM_SOCIAL     <chr> "BB RENDA FIXA CURTO PRAZO AUTOMÁTICO EMPRESA FUNDO …
$ DT_REG           <date> 2003-04-30, NA, 2003-04-30, 2003-04-30, NA, 2003-04…
$ DT_CONST         <date> 1994

In [107]:
head(relacao_fundos)

ident_ativo,nm_ativo,eh_fundo_vedado,nm_fundo,cnpj_admin,cnpj_gestor,enquad_sprev,enquad_sprev_pad,atende_art15,TP_FUNDO,⋯,DT_INI_SIT,DT_INI_EXERC,DT_FIM_EXERC,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,⋯,<date>,<date>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
00071477000168,BB RF CP AUTOMATICO,0,NA,NA,NA,NA,NA,NA,FACFIF,⋯,2005-03-24,NA,NA,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,NA,NA,NA,NA,NA
00360305000104,CAIXA FI BRASIL IRF-M 1 TP RF,0,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
00756851000169,BB ÁGIL FIC RENDA FIXA REFERENCIADO DI,0,BB RENDA FIXA REFERENCIADO DI ÁGIL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1,FACFIF,⋯,2005-03-24,NA,NA,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,NA,NA,NA,NA,NA
00812433000141,BNB AUTOMATICO FI RF CURTO PRAZO,0,BNB AUTOMÁTICO FUNDO DE INVESTIMENTO RENDA FIXA CURTO PRAZO,62318407,07237373,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1,FIF,⋯,2005-03-21,2004-04-01,2005-03-31,07.237.373/0001-20,BANCO DO NORDESTE DO BRASIL SA,NA,NA,NA,NA,NA
00822059000165,BB AÇÕES IBOVESPA ATIVO FIC,0,BB AÇÕES IBOVESPA ATIVO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 8º, Inciso I, 'a'",ART8IA,1,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
00832435000100,ITAÚ INSTITUCIONAL RENDA FIXA REFERENCIADO DI - FI,0,ITAÚ INSTITUCIONAL RENDA FIXA REFERENCIADO DI - FUNDO DE INVESTIMENTO,60701190,60701190,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1,FIF,⋯,2005-03-07,NA,NA,17.192.451/0001-70,BANCO ITAUCARD S.A.,NA,NA,NA,NA,NA


A base de dados **`relacao_fundos`** contem os fundos de investimentos nos quais ao menos um RPPS constante da base de dados do DAIR possui recursos aplicados. 

Vamos incluir na base de dados a quantidade de RPPS que possuem recursos aplicados em cada fundo de investimento. Esta informação está contida na base de dados `rpps_invest`.

In [108]:
relacao_fundos <- relacao_fundos %>%
                      left_join(rpps_invest, by="ident_ativo")
head(relacao_fundos)

ident_ativo,nm_ativo,eh_fundo_vedado,nm_fundo,cnpj_admin,cnpj_gestor,enquad_sprev,enquad_sprev_pad,atende_art15,TP_FUNDO,⋯,DT_INI_EXERC,DT_FIM_EXERC,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,qtd_rpps_investindo
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,⋯,<date>,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
00071477000168,BB RF CP AUTOMATICO,0,NA,NA,NA,NA,NA,NA,FACFIF,⋯,NA,NA,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,NA,NA,NA,NA,NA,1
00360305000104,CAIXA FI BRASIL IRF-M 1 TP RF,0,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,7
00756851000169,BB ÁGIL FIC RENDA FIXA REFERENCIADO DI,0,BB RENDA FIXA REFERENCIADO DI ÁGIL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1,FACFIF,⋯,NA,NA,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,NA,NA,NA,NA,NA,2
00812433000141,BNB AUTOMATICO FI RF CURTO PRAZO,0,BNB AUTOMÁTICO FUNDO DE INVESTIMENTO RENDA FIXA CURTO PRAZO,62318407,07237373,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1,FIF,⋯,2004-04-01,2005-03-31,07.237.373/0001-20,BANCO DO NORDESTE DO BRASIL SA,NA,NA,NA,NA,NA,2
00822059000165,BB AÇÕES IBOVESPA ATIVO FIC,0,BB AÇÕES IBOVESPA ATIVO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 8º, Inciso I, 'a'",ART8IA,1,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,6
00832435000100,ITAÚ INSTITUCIONAL RENDA FIXA REFERENCIADO DI - FI,0,ITAÚ INSTITUCIONAL RENDA FIXA REFERENCIADO DI - FUNDO DE INVESTIMENTO,60701190,60701190,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1,FIF,⋯,NA,NA,17.192.451/0001-70,BANCO ITAUCARD S.A.,NA,NA,NA,NA,NA,31


Agora vamos criar uma nova variável para indicar quais fundos estão na lista de fundos investigados.

In [109]:
relacao_fundos <- relacao_fundos %>%
                    mutate(eh_investigado = ifelse(ident_ativo %in% fundos_investigados, 1, 0))

head(relacao_fundos)

ident_ativo,nm_ativo,eh_fundo_vedado,nm_fundo,cnpj_admin,cnpj_gestor,enquad_sprev,enquad_sprev_pad,atende_art15,TP_FUNDO,⋯,DT_FIM_EXERC,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,qtd_rpps_investindo,eh_investigado
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,⋯,<date>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>
00071477000168,BB RF CP AUTOMATICO,0,NA,NA,NA,NA,NA,NA,FACFIF,⋯,NA,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,NA,NA,NA,NA,NA,1,0
00360305000104,CAIXA FI BRASIL IRF-M 1 TP RF,0,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,7,0
00756851000169,BB ÁGIL FIC RENDA FIXA REFERENCIADO DI,0,BB RENDA FIXA REFERENCIADO DI ÁGIL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1,FACFIF,⋯,NA,30.822.936/0001-69,BB GESTAO DE RECURSOS DTVM S.A,NA,NA,NA,NA,NA,2,0
00812433000141,BNB AUTOMATICO FI RF CURTO PRAZO,0,BNB AUTOMÁTICO FUNDO DE INVESTIMENTO RENDA FIXA CURTO PRAZO,62318407,07237373,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1,FIF,⋯,2005-03-31,07.237.373/0001-20,BANCO DO NORDESTE DO BRASIL SA,NA,NA,NA,NA,NA,2,0
00822059000165,BB AÇÕES IBOVESPA ATIVO FIC,0,BB AÇÕES IBOVESPA ATIVO FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 8º, Inciso I, 'a'",ART8IA,1,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,6,0
00832435000100,ITAÚ INSTITUCIONAL RENDA FIXA REFERENCIADO DI - FI,0,ITAÚ INSTITUCIONAL RENDA FIXA REFERENCIADO DI - FUNDO DE INVESTIMENTO,60701190,60701190,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1,FIF,⋯,NA,17.192.451/0001-70,BANCO ITAUCARD S.A.,NA,NA,NA,NA,NA,31,0


Vamos agora importar dados das transações diárias disponíveis no site de dados abertos da CVM: http://dados.cvm.gov.br/dataset/fi-doc-inf_diario


In [110]:
trans2020_05 <- read_delim("http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202005.csv",
                            delim = ";",
                            locale = locale(decimal_mark = ".",
                                            grouping_mark = ""))


Parsed with column specification:
cols(
  CNPJ_FUNDO = col_character(),
  DT_COMPTC = col_date(format = ""),
  VL_TOTAL = col_double(),
  VL_QUOTA = col_double(),
  VL_PATRIM_LIQ = col_double(),
  CAPTC_DIA = col_double(),
  RESG_DIA = col_double(),
  NR_COTST = col_double()
)



In [111]:
glimpse(trans2020_05)

Rows: 243,487
Columns: 8
$ CNPJ_FUNDO    <chr> "00.017.024/0001-53", "00.017.024/0001-53", "00.017.024…
$ DT_COMPTC     <date> 2020-05-04, 2020-05-05, 2020-05-06, 2020-05-07, 2020-0…
$ VL_TOTAL      <dbl> 1124995, 1125158, 1125317, 1125478, 1125299, 1125428, 1…
$ VL_QUOTA      <dbl> 27.42568, 27.42878, 27.43193, 27.43498, 27.43729, 27.43…
$ VL_PATRIM_LIQ <dbl> 1119404, 1119530, 1119659, 1119784, 1119878, 1119971, 1…
$ CAPTC_DIA     <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0…
$ RESG_DIA      <dbl> 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0…
$ NR_COTST      <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7660, 7660, 7…


In [112]:
head(trans2020_05)

CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
00.017.024/0001-53,2020-05-04,1124995,27.42568,1119404,0,0,1
00.017.024/0001-53,2020-05-05,1125158,27.42878,1119530,0,0,1
00.017.024/0001-53,2020-05-06,1125317,27.43193,1119659,0,0,1
00.017.024/0001-53,2020-05-07,1125478,27.43498,1119784,0,0,1
00.017.024/0001-53,2020-05-08,1125299,27.43729,1119878,0,0,1
00.017.024/0001-53,2020-05-11,1125428,27.43957,1119971,0,0,1


Para efeito de juntar à nossa base de dados as variáveis `NR_COTST` e `VL_PATRIM_LIQ` vamos pegar os registros relativos à data mais recente. 

In [113]:
trans2020_05_tmp <- trans2020_05 %>%
                        mutate(CNPJ_FUNDO = str_remove_all(CNPJ_FUNDO, "[[:punct:]]")) %>%
                        filter(DT_COMPTC == max(DT_COMPTC)) %>%
                        select(CNPJ_FUNDO, DT_COMPTC, VL_PATRIM_LIQ, NR_COTST)

head(trans2020_05_tmp)

CNPJ_FUNDO,DT_COMPTC,VL_PATRIM_LIQ,NR_COTST
<chr>,<date>,<dbl>,<dbl>
00017024000153,2020-05-21,1120720,1
00068305000135,2020-05-21,52367246,7648
00071477000168,2020-05-21,28124448644,318889
00073041000108,2020-05-21,19443285,969
00083181000167,2020-05-21,12392080061,4
00089915000115,2020-05-21,14783172,155


In [0]:
relacao_fundos <- relacao_fundos %>%
                    left_join(trans2020_05_tmp, by=c("ident_ativo" = "CNPJ_FUNDO"))

In [115]:
head(relacao_fundos, 3)
glimpse(relacao_fundos)

ident_ativo,nm_ativo,eh_fundo_vedado,nm_fundo,cnpj_admin,cnpj_gestor,enquad_sprev,enquad_sprev_pad,atende_art15,TP_FUNDO,⋯,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,qtd_rpps_investindo,eh_investigado,DT_COMPTC,VL_PATRIM_LIQ,NR_COTST
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<date>,<dbl>,<dbl>
00071477000168,BB RF CP AUTOMATICO,0,NA,NA,NA,NA,NA,NA,FACFIF,⋯,NA,NA,NA,NA,NA,1,0,2020-05-21,28124448644,318889
00360305000104,CAIXA FI BRASIL IRF-M 1 TP RF,0,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,7,0,NA,NA,NA
00756851000169,BB ÁGIL FIC RENDA FIXA REFERENCIADO DI,0,BB RENDA FIXA REFERENCIADO DI ÁGIL FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO,30822936,30822936,"Artigo 7º, Inciso IV, 'a'",ART7IVA,1,FACFIF,⋯,NA,NA,NA,NA,NA,2,0,2020-05-21,14176405416,420083


Rows: 656
Columns: 30
$ ident_ativo         <chr> "00071477000168", "00360305000104", "007568510001…
$ nm_ativo            <chr> "BB RF CP AUTOMATICO", "CAIXA FI BRASIL IRF-M 1 T…
$ eh_fundo_vedado     <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ nm_fundo            <chr> NA, NA, "BB RENDA FIXA REFERENCIADO DI ÁGIL FUNDO…
$ cnpj_admin          <chr> NA, NA, "30822936", "62318407", "30822936", "6070…
$ cnpj_gestor         <chr> NA, NA, "30822936", "07237373", "30822936", "6070…
$ enquad_sprev        <chr> NA, NA, "Artigo 7º, Inciso IV, 'a'", "Artigo 7º, …
$ enquad_sprev_pad    <chr> NA, NA, "ART7IVA", "ART7IVA", "ART8IA", "ART7IVA"…
$ atende_art15        <dbl> NA, NA, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, NA, 1, 1…
$ TP_FUNDO            <chr> "FACFIF", NA, "FACFIF", "FIF", NA, "FIF", "FIF", …
$ DENOM_SOCIAL        <chr> "BB RENDA FIXA CURTO PRAZO AUTOMÁTICO EMPRESA FUN…
$ DT_REG              <date> 2003-04-30, NA, 2003-04-30, 2003-04-30, NA, 2003…
$ DT_CONST            <date> 1

Pronto. Chegamos á nossa base de dados final. Agora é começar a explorá-la para ver se sai alguma coisa interessante.

<br>
<br>

---

# **PARTE II - CÁLCULO DA RENTABILIDADE DA CARTEIRA DE FUNDOS DE INVESTIMENTOS**

---

<br>

Nesta parte do documento, nosso objetivo é estruturar uma metodologia para calcular a rentabilidade de uma carteira de fundos de investimentos de um RPPS.

<br>
<br>

---

# **PARTE III - ANÁLISE FUNDOS DE INVESTIMENTOS DE RPPS DO RJ**

---

<br>

Vamos agora restringir a base de dados para examinar apenas os registros relativos a RPPS do Estado do Rio de Janeiro.

In [0]:
dair_fundos_rj <- dair_fundos %>%
                    filter(uf == "RJ")

# criar uma lista contendo os rpps de um ente

In [0]:
fundos_representacao_sprev_rj <- c(12845801000137, # 
                                   13344834000166,
                                   13594673000169,
                                   12845801000137,
                                   14655180000154,
                                   18373362000193,
                                   15798354000109)


**Quais RPPS possuem recursos aplicados nos fundos elencados acima?**

In [0]:
entes_fundos_ruins <- dair_fundos_rj %>%
                          filter(ident_ativo %in% fundos_representacao_sprev_rj) %>%
                          distinct(ident_ativo, nm_ativo, ente, tipo_ativo, ) %>%
                          arrange(ident_ativo)

In [46]:
entes_fundos_ruins

ident_ativo,nm_ativo,ente,tipo_ativo
<chr>,<chr>,<chr>,<chr>
12845801000137,TOWER BRIDGE FI RF IMAB 5,Japeri,"ETF - Renda Fixa ""Referenciado"" - Art. 7º, III, b"
12845801000137,TOWER BRIDGE RF FI IMA-B 5,Japeri,"FI Renda Fixa - Geral - Art. 7º, IV, a"
12845801000137,TOWER BRIDGE IMA-B 5 FI RENDA FIXA,Queimados,"FI Renda Fixa ""Referenciado"" - Art. 7º, III, a"
12845801000137,TOWER BRIDGE IMA-B 5 FI RENDA FIXA,Queimados,"FI Renda Fixa - Geral - Art. 7º, IV, a"
13344834000166,INCENTIVO FI em DC MULTISETORIAL II,Japeri,"FI em Direitos Creditórios - Cota Sênior - Art. 7º, VII, a"
13594673000169,TMJ IMA B FI RF,Japeri,"FI Renda Fixa - Geral - Art. 7º, IV, a"
13594673000169,TMJ IMA B FI RF,Japeri,"ETF - Renda Fixa ""Referenciado"" - Art. 7º, III, b"
14655180000154,FI MULTIMERCADO SCULPTOR CRÉDITO PRIVADO,Belford Roxo,"FI Multimercado - Aberto - Art. 8º, III"
15798354000109,FIP LSH MULTIESTRATÉGIA,Iguaba Grande,"FI em Participações - Art. 8º, IV, a"
